NOTES FOR LAB EXAM :DDDD!!!!!

ALL IMPORTS OF THE WORLD

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy.stats import ttest_ind
from scipy.stats import shapiro, mannwhitneyu, levene, kruskal
from scipy.stats import chi2_contingency, fisher_exact
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import ks_2samp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.power import TTestIndPower

PROPORTION AKA CHI-SQUARE TEST WITH CONTINGENCY TABLE

In [ ]:
# Firstly get the data we need
higher_rank = salaries[(salaries['rank'] == 'AssocProf') | (salaries['rank'] == 'Prof')]

# Since its proportion do contingency table of the rows, columns needed
contingency_table = pd.crosstab(higher_rank['sex'], higher_rank['rank'])
print("Contingency Table:")
print(contingency_table)


# Perform Chi-square
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)
print("\nChi-Square Test Results:")
print("Chi-Square Statistic =", chi2_stat)
print("P-value =", p_value)
print("Degrees of Freedom =", dof)
print("Expected Frequencies:")
print(expected)

# 4. Fisher's Exact Test (for small sample sizes)
if (expected < 5).any():
    print("\nExpected frequencies are too small. Using Fisher's Exact Test.")
    _, fisher_p_value = fisher_exact(contingency_table)
    print("Fisher's Exact Test P-value =", fisher_p_value)
else:
    print("\nChi-Square test is valid. No need for Fisher's Exact Test.")

# Interpretation of Results
if p_value < 0.05:
    print("\nConclusion: We reject the null hypothesis.")
else:
    print("\nConclusion: We fail to reject the null hypothesis.")


CLASSIC T-TEST OR MANN-WHITNEY RANK TEST BASED ON NORMALITY FROM SHAPIRO

In [3]:
# Filter data
theoretical_salaries = salaries[salaries['discipline'] == 'B']['salary']
applied_salaries = salaries[salaries['discipline'] == 'A']['salary']

# 1. Normality Test (Shapiro-Wilk Test)
print("Shapiro-Wilk Test for Normality:")
shapiro_theoretical = shapiro(theoretical_salaries)   # From the data we got
shapiro_applied = shapiro(applied_salaries)

print("Theoretical (B): W-statistic =", shapiro_theoretical.statistic, ", P-value =", shapiro_theoretical.pvalue)
print("Applied (A): W-statistic =", shapiro_applied.statistic, ", P-value =", shapiro_applied.pvalue)

# 2. Choose the Appropriate Test
if shapiro_theoretical.pvalue >= 0.05 and shapiro_applied.pvalue >= 0.05:
    print("\nBoth groups are normally distributed. Proceeding with T-test.")
    # Perform T-test
    t_stat, p_value = ttest_ind(theoretical_salaries, applied_salaries, equal_var=False)  # Welch's T-test
    print("\nT-Test Results:")
    print("T-statistic =", t_stat)
    print("P-value =", p_value)
    
    if p_value < 0.05:
        print("We reject the null hypothesis: There is a significant difference in salaries.")
    else:
        print("We fail to reject the null hypothesis: There is no significant difference in salaries.")
else:
    print("\nAt least one group is NOT normally distributed. Proceeding with Mann-Whitney U Test.")
    # Perform Mann-Whitney U Test
    u_stat, p_value = mannwhitneyu(theoretical_salaries, applied_salaries, alternative='two-sided')
    print("\nMann-Whitney U Test Results:")
    print("U-statistic =", u_stat)
    print("P-value =", p_value)
    
    if p_value < 0.05:
        print("We reject the null hypothesis: There is a significant difference in salaries.")
    else:
        print("We fail to reject the null hypothesis: There is no significant difference in salaries.")

3-WAY ANOVA

In [ ]:
# Read the exercise 
# dependent variable - what we want to check e.g salary
# independent variable - based on what

# Convert categorical variables to category data type so that there is no incorrect Anova behavior
salaries['rank'] = salaries['rank'].astype('category')
salaries['sex'] = salaries['sex'].astype('category')
salaries['discipline'] = salaries['discipline'].astype('category')

# Specify the ANOVA model
model = ols('salary ~ rank * sex * discipline', data=salaries).fit()

# ASSUMPTIONS FOR ANOVA
# Normality
residuals = model.resid
# Perform Shapiro-Wilk test for normality
shapiro_test = shapiro(residuals)
print("Shapiro-Wilk Test:")
print(f"W-statistic = {shapiro_test.statistic}, p-value = {shapiro_test.pvalue}")

plt.subplot(1, 2, 2)
sm.qqplot(residuals, line='s', fit=True)
plt.title("Q-Q Plot")
plt.tight_layout()
plt.show()

# Homogenity of variance
levene_test = levene(
    salaries[salaries['rank'] == 'Prof']['salary'],
    salaries[salaries['rank'] == 'AssocProf']['salary'],
    salaries[salaries['rank'] == 'AsstProf']['salary']
)
print("\nLevene's Test:")
print(f"Statistic = {levene_test.statistic}, p-value = {levene_test.pvalue}")

# If ok do normal anova
anova_results = anova_lm(model, typ=2)  # Type II ANOVA table
print(anova_results)

# If no homogenity of variance
# Perform Welch's ANOVA
welch_anova = pg.welch_anova(dv='salary', between=['rank', 'sex', 'discipline'], data=salaries)
print("\nWelch's ANOVA Results:")
print(welch_anova)

GOODNESS OF FIT (KOLMOGOROV-SMIRNOV) for 2 samples

In [ ]:
ks_stat, ks_p_value = ks_2samp(male_credit['amount'], female_credit['amount'])

print("\nKolmogorov-Smirnov Test:")
print(f"KS Statistic = {ks_stat}")
print(f"P-value = {ks_p_value}")

if ks_p_value < 0.05:
    print("The distributions of credit amounts for males and females are significantly different.")
else:
    print("The distributions of credit amounts for males and females are not significantly different.")


EFFECT SIZE AND POWER

In [ ]:
# ---------------------
# Step 1: Calculate Effect Size (Cohen's d)
# ---------------------
# Separate total_bill amounts for smokers and non-smokers
smokers = tips[tips['smoker'] == 'Yes']['total_bill']  #GET DATASET
non_smokers = tips[tips['smoker'] == 'No']['total_bill']

# Compute means and standard deviations
mean_smokers = smokers.mean()
mean_non_smokers = non_smokers.mean()
std_smokers = smokers.std()
std_non_smokers = non_smokers.std()

# Pooled standard deviation
n_smokers = len(smokers)
n_non_smokers = len(non_smokers)
pooled_std = np.sqrt(((n_smokers - 1) * std_smokers**2 + (n_non_smokers - 1) * std_non_smokers**2) /
                     (n_smokers + n_non_smokers - 2))

# Cohen's d
effect_size = (mean_smokers - mean_non_smokers) / pooled_std
print(f"Effect Size (Cohen's d): {effect_size:.4f}")

# ---------------------
# Step 2: Calculate Statistical Power
# ---------------------
# Define the test parameters
alpha = 0.05  # Significance level
analysis = TTestIndPower()
power = analysis.power(effect_size=effect_size, nobs1=n_smokers, alpha=alpha, ratio=n_non_smokers / n_smokers)

print(f"Power of the test: {power:.4f}")

# ---------------------
# Step 3: Determine Required Sample Size for 90% Power
# ---------------------
# Calculate the required sample size for 90% power
required_n = analysis.solve_power(effect_size=effect_size, power=0.90, alpha=alpha, ratio=n_non_smokers / n_smokers)
print(f"Required sample size per group for 90% power: {np.ceil(required_n):.0f}")

PAIRED T-TEST

In [ ]:
import numpy as np
import scipy.stats as stats

# Sample data
before = np.array([85, 78, 92, 88, 76])
after = np.array([80, 75, 89, 86, 78])

# Paired t-test
t_stat, p_value_t = stats.ttest_rel(before, after)
print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value_t}")

WILCOXON PAIRED

In [ ]:
import numpy as np
import scipy.stats as stats

# Sample data
before = np.array([85, 78, 92, 88, 76])
after = np.array([80, 75, 89, 86, 78])

# Wilcoxon signed-rank test
w_stat, p_value_w = stats.wilcoxon(before, after)
print(f"Wilcoxon signed-rank test: W-statistic = {w_stat}, p-value = {p_value_w}")